In [4]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

INPUT_DIR = "./data"
FILENAME = ["浦发银行", "浙江龙盛", "上证综指", "国债指数"]

df = pd.read_csv(os.path.join(INPUT_DIR, "month_data.csv"))
print(df.head())

rate_of_ret = {}
for filename in FILENAME:
    rate_of_ret[filename] = np.array(df[df["name"] == filename]["rate_of_ret"])

excess_income = {}
for filename in FILENAME:
    excess_income[filename] = rate_of_ret[filename] - rate_of_ret["国债指数"]

   Unnamed: 0  name     date  rate_of_ret  rate_of_ret_accumulate
0           0  浦发银行  2016-05     0.019175                0.019175
1           1  浦发银行  2016-06    -0.035748               -0.017258
2           2  浦发银行  2016-07     0.007805               -0.009588
3           3  浦发银行  2016-08     0.050339                0.040268
4           4  浦发银行  2016-09     0.000000                0.040268


In [10]:
def get_OLS(filename):
    x = excess_income["上证综指"] / 100.
    y = excess_income[filename] / 100. 
    X = sm.add_constant(x)
    model = sm.OLS(y, X)
    model = model.fit()
    return model 

model1 = get_OLS("浦发银行")
model2 = get_OLS("浙江龙盛")

print("model: 浦发银行")
print(model1.summary())
print("\nmodel: 浙江龙盛")
print(model2.summary())

model: 浦发银行
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     23.76
Date:                Sat, 13 Nov 2021   Prob (F-statistic):           8.85e-06
Time:                        12:44:43   Log-Likelihood:                 376.28
No. Observations:                  60   AIC:                            -748.6
Df Residuals:                      58   BIC:                            -744.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.277e-05      6e-05     -0.

In [17]:
X1 = sm.add_constant(excess_income["浦发银行"])
y_hat1 = model1.predict(X1)
X2 = sm.add_constant(excess_income["浙江龙盛"])
y_hat2 = model2.predict(X2)

y_error1 = excess_income["浦发银行"] - y_hat1
y_error2 = excess_income["浙江龙盛"] - y_hat2

total_risk = [excess_income["浦发银行"].std(), excess_income["浙江龙盛"].std()]
system_risk = [y_hat1.std(), y_hat2.std()]
company_risk = [y_error1.std(), y_error2.std()]

df = pd.DataFrame()
df["股票"] = ["浦发银行", "浙江龙盛"]
df["总风险"] = total_risk
df["系统性风险"] = system_risk
df["公司风险"] = company_risk

print(df.head())




     股票       总风险     系统性风险      公司风险
0  浦发银行  0.054287  0.039169  0.015119
1  浙江龙盛  0.109385  0.148096  0.038711
